# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:

df = spark.read.option("header", "true") \
               .option("delimiter", ";") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .csv("dbfs:/FileStore/ds_jobs_in_.csv")
df.printSchema()
df.show()

2. Eliminar duplicados

Comprobamos los duplicados

In [0]:
# Contar filas duplicadas (todas las columnas)
df.groupBy(df.columns).count().filter("count > 1").display()


index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,count


In [0]:
df_sin_duplicados = df.dropDuplicates()
df_sin_duplicados.show()


+-----+--------------------+--------------------+--------------------+------+--------------------+-----------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|           Job Title|     Salary Estimate|     Job Description|Rating|        Company Name|         Location|        Headquarters|                Size|Founded|   Type of ownership|            Industry|              Sector|             Revenue|         Competitors|
+-----+--------------------+--------------------+--------------------+------+--------------------+-----------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   19|Medical Lab Scien...|$137K-$171K (Glas...|Responsibilities\...|    35|   Tower Health\n3.5|   West Grove, PA|         Reading, PA|5001 to 10000 emp...|   2017|Nonprofit Organiz.

3. Decidir que hacer con los datos faltantes 

Como podemos comprobar hay una fila con valores nulos por lo que procedemos a eliminarla ya que solo es 1 fila por lo que no nos va a aportar mucha informacion

4. Decidir que hacer con los valores nulos

In [0]:
from pyspark.sql.functions import col, isnan, when, count

# Contar valores nulos o NaN por columna
valores_nulos = df_sin_duplicados.select(
    [count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]
)
valores_nulos.display()

index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:

df_sin_nulos = df.dropna()


In [0]:
from functools import reduce

# Contar filas con valores nulos
filas_con_nulos = df_sin_nulos.filter(
    reduce(lambda a, b: a | b, (col(c).isNull() for c in df_sin_nulos.columns))
).count()

if filas_con_nulos == 0:
    print("No hay valores nulos en el DataFrame.")
else:
    print(f"Hay {filas_con_nulos} filas con valores nulos.")


No hay valores nulos en el DataFrame.


5. ¿Cuántos registros tiene el csv?

In [0]:
# Contar el número de registros en el DataFrame
numero_registros = df.count()
print(f"El archivo CSV tiene {numero_registros} registros.")


El archivo CSV tiene 678 registros.


6. Mostrar los valores únicos de `Job title` 

In [0]:
# Mostrar los valores únicos de la columna 'Job title'
df.select("Job title").distinct().show(truncate=False)


+----------------------------------------------+
|Job title                                     |
+----------------------------------------------+
|Business Intelligence Analyst                 |
|Data Modeler                                  |
|Senior Research Statistician- Data Scientist  |
|Sr Data Scientist                             |
|Data Scientist/Machine Learning               |
|Data Scientist / Machine Learning Expert      |
|Associate Data Scientist                      |
|Medical Lab Scientist                         |
|Human Factors Scientist                       |
|Experienced Data Scientist                    |
|Data Analyst II                               |
|Data Scientist                                |
|Data Analyst                                  |
|Senior Analyst/Data Scientist                 |
|Data Scientist-Human Resources                |
|Data Scientist - Contract                     |
|Data Scientist - Risk                         |
|Business Intelligen

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, split, trim, expr, col, concat_ws

df = df.withColumn("Salary Estimate", regexp_replace(col("Salary Estimate"), "K", ".000"))

# Mostrar el resultado final
df.show()





8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
# Mostrar los valores únicos de la columna "Salary Estimate"
valores_unicos = df.select("Salary Estimate").distinct()

# Mostrar los resultados
valores_unicos.show(truncate=False)

+----------------------------------+
|Salary Estimate                   |
+----------------------------------+
|$75.000-$131.000 (Glassdoor est.) |
|$90.000-$109.000 (Glassdoor est.) |
|$141.000-$225.000 (Glassdoor est.)|
|$101.000-$165.000 (Glassdoor est.)|
|$110.000-$163.000 (Glassdoor est.)|
|$79.000-$106.000 (Glassdoor est.) |
|$99.000-$132.000 (Glassdoor est.) |
|$122.000-$146.000 (Glassdoor est.)|
|$124.000-$198.000 (Glassdoor est.)|
|$137.000-$171.000 (Glassdoor est.)|
|$112.000-$116.000 (Glassdoor est.)|
|$145.000-$225.000(Employer est.)  |
|$79.000-$131.000 (Glassdoor est.) |
|$90.000-$124.000 (Glassdoor est.) |
|$71.000-$123.000 (Glassdoor est.) |
|$91.000-$150.000 (Glassdoor est.) |
|$69.000-$116.000 (Glassdoor est.) |
|$79.000-$147.000 (Glassdoor est.) |
|$56.000-$97.000 (Glassdoor est.)  |
|$31.000-$56.000 (Glassdoor est.)  |
+----------------------------------+
only showing top 20 rows



9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col



# Eliminar (Glassdoor est.) y (Employer est.) de la columna Salary Estimate
df = df.withColumn(
    "Salary Estimate",
    regexp_replace(
        regexp_replace(col("Salary Estimate"), r"\s*\(Glassdoor est.\)", ""),  # Eliminar (Glassdoor est.)
        r"\s*\(Employer est.\)", ""  # Eliminar (Employer est.)
    )
)

# Mostrar el DataFrame después de la limpieza
df.show()

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
# Supongamos que ya tienes el DataFrame df creado
df_salary= df.select("Salary Estimate").orderBy(col("Salary Estimate").desc()).show(truncate=False)



11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
# Quitar los saltos de línea \n de la columna "Job Description"
df = df.withColumn("Job Description", regexp_replace(col("Job Description"), "\\n", " "))
df.show()


12. De la columna `Rating` muestre los valores unicos.

In [0]:
# Mostrar los valores únicos de la columna "Rating"
unique_ratings = df.select("Rating").distinct()

# Mostrar los resultados
unique_ratings.show(truncate=False)

+------+
|Rating|
+------+
|29    |
|42    |
|34    |
|28    |
|22    |
|35    |
|47    |
|43    |
|31    |
|27    |
|41    |
|38    |
|44    |
|33    |
|48    |
|32    |
|36    |
|37    |
|39    |
|50    |
+------+
only showing top 20 rows



13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
# Reemplazar -1.0 por 0.0 en la columna "Rating"
df = df.withColumn("Rating", when(col("Rating") == -1.0, 0.0).otherwise(col("Rating")))


14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
# Mostrar los valores únicos de la columna "Company Name" y ordenarlos
unique_company_names = df.select("Company Name").distinct().orderBy("Company Name")

# Mostrar los resultados
unique_company_names.show(truncate=False)

15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
# Limpiar caracteres innecesarios de la columna "Company Name"
# Reemplaza saltos de línea, tabulaciones, signos de exclamación y espacios en blanco al principio y al final
df_cleaned = df.withColumn("Company Name", 
    regexp_replace(col("Company Name"), "[\\n\\t!]", "")  # Reemplaza \n, \t y !
)

# Limpiar espacios en blanco adicionales
df_cleaned = df_cleaned.withColumn("Company Name", 
    regexp_replace(col("Company Name"), " +", " ")  # Reemplaza múltiples espacios por uno solo
)

# Eliminar espacios en blanco al principio y al final
df= df_cleaned.withColumn("Company Name", 
    regexp_replace(col("Company Name"), "^\\s+|\\s+$", "")  # Quita espacios al inicio y al final
)

# Mostrar el DataFrame después de la limpieza
df.show(truncate=False)

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
# Dividir la columna "Location" en "City" y "State"
df = df.withColumn("City", split(col("Location"), ",")[0]) \
              .withColumn("State", split(col("Location"), ",")[1]) \
              .drop("Location")  # Eliminar la columna "Location"

# Mostrar el DataFrame después de la división

df.show(truncate=False)

+-----+-----------------------------------------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
# Dividir la columna "Headquarters" en "Headquarter City" y "Headquarter State"
df = df.withColumn("Headquarter City", split(col("Headquarters"), ",")[0]) \
              .withColumn("Headquarter State", split(col("Headquarters"), ",")[1]) \
              .drop("Headquarters")  # Eliminar la columna "Headquarters"

# Mostrar el DataFrame después de la división
df.show()

+-----+--------------------+-----------------+--------------------+------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|index|           Job Title|  Salary Estimate|     Job Description|Rating|        Company Name|                Size|Founded|   Type of ownership|            Industry|              Sector|             Revenue|         Competitors|         City|State|Headquarter City|Headquarter State|
+-----+--------------------+-----------------+--------------------+------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|    0|   Sr Data Scientist|$137.000-$171.000|Description  The ...|    31|      Healthfirst3.1|1001 to 5000 empl...|   1993|Nonprofit Organiz...|

18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
# Obtener los valores únicos de la columna "Headquarter State"
unique_headquarter_states = df.select("Headquarter State").distinct()

unique_headquarter_states.display()

Headquarter State
WA
AL
NM
MI
HI
MO
Israel
Japan
NE
null


19. Mostrar valores unicos del campo `Size`.

In [0]:
# Obtener los valores únicos de la columna "Headquarter State"
unique_size = df.select("Size").distinct()

unique_size.display()

Size
-1
5001 to 10000 employees
0
Unknown
51 to 200 employees
1001 to 5000 employees
501 to 1000 employees
201 to 500 employees
10000+ employees
1 to 50 employees


20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
# Limpiar la columna "Size"
df_cleaned = df.withColumn("Size", 
    regexp_replace(col("Size"), " employees|Unknown|-1", "")  # Reemplaza 'employees', 'Unknown' y '-1'
)

# Limpiar espacios en blanco adicionales
df_cleaned = df_cleaned.withColumn("Size", 
    regexp_replace(col("Size"), " +", " ")  # Reemplaza múltiples espacios por uno solo
)

# Eliminar espacios en blanco al principio y al final
df = df_cleaned.withColumn("Size", 
    regexp_replace(col("Size"), "^\\s+|\\s+$", "")  # Quita espacios al inicio y al final
)

# Mostrar el DataFrame después de la limpieza
df.show()

+-----+--------------------+-----------------+--------------------+------+--------------------+-------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|index|           Job Title|  Salary Estimate|     Job Description|Rating|        Company Name|         Size|Founded|   Type of ownership|            Industry|              Sector|             Revenue|         Competitors|         City|State|Headquarter City|Headquarter State|
+-----+--------------------+-----------------+--------------------+------+--------------------+-------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|    0|   Sr Data Scientist|$137.000-$171.000|Description  The ...|    31|      Healthfirst3.1| 1001 to 5000|   1993|Nonprofit Organiz...|  Insurance Carriers|       

21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
# Reemplazar 'to' por '-' y '-1' por 'Unknown'
df_replaced = df.withColumn("Size", 
    regexp_replace(col("Size"), "to", "-"))  # Reemplazar 'to' por '-'

df= df_replaced.withColumn("Size", 
    regexp_replace(col("Size"), "-1", "Unknown"))  # Reemplazar '-1' por 'Unknown'

# Mostrar el DataFrame después de los reemplazos
df.show()

+-----+--------------------+-----------------+--------------------+------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|index|           Job Title|  Salary Estimate|     Job Description|Rating|        Company Name|        Size|Founded|   Type of ownership|            Industry|              Sector|             Revenue|         Competitors|         City|State|Headquarter City|Headquarter State|
+-----+--------------------+-----------------+--------------------+------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|    0|   Sr Data Scientist|$137.000-$171.000|Description  The ...|    31|      Healthfirst3.1| 1001 - 5000|   1993|Nonprofit Organiz...|  Insurance Carriers|           

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
df.printSchema()  # Mostrar el esquema

# Obtener los valores únicos de la columna "Type of ownership"
unique_ownership_types = df.select("Type of ownership").distinct()

# Mostrar los resultados de los valores únicos
unique_ownership_types.display()

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter City: string (nullable = true)
 |-- Headquarter State: string (nullable = true)



Type of ownership
-1
Government
Subsidiary or Business Segment
Self-employed
Contract
Unknown
College / University
Company - Private
Nonprofit Organization
Hospital


23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
# Reemplazar '-1' por 'Unknown'
df = df.withColumn("Type of ownership", 
    when(col("Type of ownership") == '-1', 'Unknown')  # Cambiar '-1' por 'Unknown'
    .otherwise(col("Type of ownership"))  # Mantener el valor original si no es '-1'
)

df.show()

+-----+--------------------+-----------------+--------------------+------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|index|           Job Title|  Salary Estimate|     Job Description|Rating|        Company Name|        Size|Founded|   Type of ownership|            Industry|              Sector|             Revenue|         Competitors|         City|State|Headquarter City|Headquarter State|
+-----+--------------------+-----------------+--------------------+------+--------------------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+----------------+-----------------+
|    0|   Sr Data Scientist|$137.000-$171.000|Description  The ...|    31|      Healthfirst3.1| 1001 - 5000|   1993|Nonprofit Organiz...|  Insurance Carriers|           

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
# Reemplazar múltiples valores en la columna "Type of ownership"
df_replaced = df.withColumn("Type of ownership", 
    regexp_replace(col("Type of ownership"), "Company - Public", "Public Company")
)

df_replaced = df_replaced.withColumn("Type of ownership", 
    regexp_replace(col("Type of ownership"), "Company - Private", "Private Company")
)

df_replaced = df_replaced.withColumn("Type of ownership", 
    regexp_replace(col("Type of ownership"), "Private Practice / Firm", "Private Company")
)

df_replaced = df_replaced.withColumn("Type of ownership", 
    regexp_replace(col("Type of ownership"), "Subsidiary or Business Segment", "Business")
)

df = df_replaced.withColumn("Type of ownership", 
    regexp_replace(col("Type of ownership"), "College / University", "Education")
)

df.display()

25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
df.printSchema()  # Mostrar el esquema del DataFrame

# Obtener los valores únicos de la columna "Industry"
unique_industry_types = df.select("Industry").distinct()

unique_industry_types.display()

26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
# Reemplazar '-1' por 'Not Available' y '&' por 'and'
df= df.withColumn("Industry", 
    regexp_replace(col("Industry"), "-1", "Not Available")) \
    .withColumn("Industry", 
    regexp_replace(col("Industry"), "&", "and"))

# Obtener los valores únicos de la columna "Industry" y ordenarlos alfabéticamente
unique_industry_types = df.select("Industry").distinct().orderBy("Industry")

unique_industry_types.display()


27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
df.printSchema()  # Mostrar el esquema del DataFrame

# Obtener los valores únicos de la columna "Sector"
unique_sector_types = df.select("Sector").distinct()

unique_sector_types.display()

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter City: string (nullable = true)
 |-- Headquarter State: string (nullable = true)



Sector
Health Care
-1
Education
Insurance
Information Technology
Government
"Oil, Gas, Energy & Utilities"
Finance
Real Estate
Non-Profit


28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
# Reemplazar '-1' por 'Not Available' y '&' por 'and' en "Sector"
df = df.withColumn("Sector", 
    regexp_replace(col("Sector"), "-1", "Not Available")) \
    .withColumn("Sector", 
    regexp_replace(col("Sector"), "&", "and"))

# Obtener los valores únicos de la columna "Sector" después de reemplazos
unique_sector_types_replaced = df.select("Sector").distinct().orderBy("Sector")

unique_sector_types_replaced.display()

Sector
Accounting and Legal
Aerospace and Defense
Agriculture and Forestry
Biotech and Pharmaceuticals
Business Services
"Construction, Repair and Maintenance"
Consumer Services
Education
Finance
Government


29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
df.printSchema()  # Mostrar el esquema del DataFrame de Revenue

# Obtener los valores únicos de la columna "Revenue"
unique_revenue_types = df.select("Revenue").distinct()

unique_revenue_types.display()


root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter City: string (nullable = true)
 |-- Headquarter State: string (nullable = true)



Revenue
-1
$50 to $100 million (USD)
$100 to $500 million (USD)
0
$10 to $25 million (USD)
$10+ billion (USD)
$1 to $5 million (USD)
$500 million to $1 billion (USD)
Unknown / Non-Applicable
$25 to $50 million (USD)


30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
df= df \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "-1", "N/A")) \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "Unknown / Non-Applicable", "N/A")) \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "Less than \\$1 million \\(USD\\)", "Less than 1")) \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "\\$", "")) \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "\\(USD\\)", "")) \
    .withColumn("Revenue", regexp_replace(col("Revenue"), "\\s+", " "))  # Quitar espacios extras

df.show()

31. Borrar el campo `Competitors`.

In [0]:
# Eliminar la columna "Competitors"
df= df.drop("Competitors")

df.display()

32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
# Extraer los salarios mínimo y máximo
df_salary = df.withColumn("min_salary", regexp_extract(col("Salary Estimate"), r"\$(\d+)", 1).cast("integer")) \
              .withColumn("max_salary", regexp_extract(col("Salary Estimate"), r"-\$(\d+)", 1).cast("integer"))


df_salary.display()

In [0]:
from pyspark.sql.functions import regexp_extract, col, round

# Extraer los salarios mínimo y máximo
df_salary = df.withColumn("min_salary", regexp_extract(col("Salary Estimate"), r"\$(\d+)", 1).cast("integer")) \
              .withColumn("max_salary", regexp_extract(col("Salary Estimate"), r"-\$(\d+)", 1).cast("integer"))


# Calcular el salario promedio
df= df_salary.withColumn("avg_salary", round((col("min_salary") + col("max_salary")) / 2, 2))

df.display()

33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
df.printSchema()  # Mostrar el esquema del DataFrame

# Obtener los valores únicos de la columna "Founded"
unique_founded_years = df.select("Founded").distinct()

unique_founded_years.display()

34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
# Reemplazar '-1' por '2024' en la columna "Founded"
df= df.withColumn("Founded", when(col("Founded") == -1, 2024).otherwise(col("Founded")))

df.display()


35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
# Calcular la edad de la empresa
df = df.withColumn("company_age", 2024 - col("Founded"))

df.display()

36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
# Crear la nueva columna "Job Type"
df = df.withColumn("Job Type",
    when(col("Job Title").rlike("(?i)sr|senior|lead|principal"), "Senior")
    .when(col("Job Title").rlike("(?i)jr|jr\\."), "Junior")
    .otherwise("NA")
)

df.display()


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
# Obtener los valores únicos de la columna "Job Type"
unique_job_types = df.select("Job Type").distinct()

unique_job_types.display()


38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


In [0]:
# Definir las habilidades a extraer
skills = ["Python", "Spark", "Big Data", "SQL", "Excel", "Machine Learning", "Data Analytics", "Visualization"]

# Crear nuevas columnas binarias para cada habilidad
for skill in skills:
    df = df.withColumn(skill, when(col("Job Description").rlike(skill), 1).otherwise(0))

df.display()

39. Exportar dataset final a csv

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

Este análisis se centra en cómo el salario y la calificación de una empresa pueden afectar la percepción de los candidatos sobre la organización. Comprender estas relaciones es crucial para que tanto las empresas como los candidatos puedan posicionarse mejor en un mercado laboral dinámico.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, year, current_date

# Calcular la antigüedad de la empresa
current_year = year(current_date())
df = df.withColumn("Company Age", current_year - col("Founded"))

# Calcular el rating promedio por tipo de propiedad
avg_rating_by_ownership = df.groupBy("Type of ownership").agg(avg("Rating").alias("Avg Rating"))

# Calcular el rating promedio por tamaño de empresa
avg_rating_by_size = df.groupBy("Size").agg(avg("Rating").alias("Avg Rating"))

# Visualizar la relación entre Antigüedad y Rating
import matplotlib.pyplot as plt

age_rating_df = df.select("Company Age", "Rating").toPandas()
plt.figure(figsize=(10, 5))
plt.scatter(age_rating_df["Company Age"], age_rating_df["Rating"], color='orange')
plt.title("Company Age vs Rating")
plt.xlabel("Company Age (Years)")
plt.ylabel("Rating")
plt.grid(True)
plt.show()

# Contar la cantidad de empresas por tipo de propiedad
ownership_distribution = df.groupBy("Type of ownership").agg(count("Company Name").alias("Count"))
